# Projet RO - Optimisation Hivernale
Dans ce projet, nous allons comprendre comment optimiser le déneigement de la ville de Montréal.

Cette démonstration comporte 3 parties:
* Analyse du problème
* Parcours du Drone
    * Théorie
    * Pratique
* Parcours de la Déneigeuse
    * Théorie
    * Pratique
* Modèle de Coût
* Conclusion

Nous allons commencer par définir le vocabulaire que nous utiliserons dans ce notebook :

* Un node : un nœud du graphe
* Un edge : l’axe qui relie deux node entre eux

En première partie nous allons nous occuper du trajet du drone. Pour cela nous considérons donc un graphe non orienté car le drone n’a pas besoin de suivre le sens de la circulation (et que c’est plus simple !). Nous avons aussi fabriqué une fonction qui vérifie si un graphe est connexe. En effet, si toutes les rues d’une ville ne sont pas reliées ce n’est pas une ville… 

On parcourt donc le graphe et on attribue à chaque edge deux attributs : son « weight », sa taille en m calculée plus tard et sa « snow » qui est le pourcentage de rue recouverte par la neige. 



In [ ]:
function


Une fois que le parcours du graphe par le drone est acté, nous pouvons passer au parcours de la déneigeuse. Celle-ci doit en revanche suivre les sens de circulation, nous allons donc avoir besoin d’un graphe orienté. Nous allons à partir d’ici utiliser la bibliothèque OSMNX qui nous permet de transformer n’importe quel plan de ville en graphe orienté grâce à l’attribut graph_from_place. Nous devons ajouter le paramètre « drive » car la vitesse de déplacement de la déneigeuse se rapproche plus de celle d’une voiture que de celle d’un passant. A partir de ce graphe, nous allons créer un graphe Eulérien. Pourquoi ? Parce que cela permet de passer par toutes les routes du graphe d’un seul coup sans passer des points plusieurs fois, en somme ça nous permet d’optimiser notre solution. Comment ? Nous allons assigner une balance à chaque node. +1 pour chaque edge sortant -1 pour chaque edge entrant. Exemple avec ce graphe simple :

(schema)

Pour un graphe orienté, il faut que toutes les balances soient nulles. On parcourt donc le graphe et à chaque balance non nulle, on ajoute des edges entrants ou sortants selon nos besoins. La fonction Distance de la librairie OSMNX nous permet d’ajouter la longueur des rues (ou des edges) en mètres sur le graphe. 

Ensuite, pour rendre tout cela plus visuel et plus attractif, on va afficher le graphe à l’écran à l’aide de la fonction plot_graph. Sur cet exemple, la ville est petite et on a donc besoin de faire qu’une seule requête au système afin d’afficher et d’utiliser le graphe. En revanche, pour Montréal, nous devrons faire une dizaine de requêtes de part la taille de la ville. Il faudra néanmoins faire attention car si on lance plusieurs requêtes en moins d’une seconde, on est banni du système pendant 60 secondes.


schema


Ce graphe n’est pas exploitable avec nos fonctions, qui elles sont compatibles avec la class Graph que nous avons construite, beaucoup plus simple et compréhensible. Nous devons donc transformer ce graphe et le rediriger vers une forme qui correspond aux attentes de nos fonctions. Cette action est possible grâce à la fonction undirectedgraphmxgraph_to_graph.

In [ ]:
function

Sur cette version simplifiée, on applique nos fonction théoriques, celles qui le parcourent, lui affectent des attributs de poids et de tailles... Une fois tous ces calculs effectués, on remet le graph sous sa forme originelle grâce à cette fonction :

In [ ]:
function

Et ça nous permet d’afficher cela :

graphe avec neige dessus

Afin d’avoir une idée du coût du déneigement total de Montréal, nous avons choisi de mesurer trois indicateurs : le temps + coût en carburant pour une seule déneigeuse, pour une déneigeuse pour 2 quartiers et enfin pour une déneigeuse par quartier.

cout1
cout2
cout3


On peut clairement voir que le coût est moindre lorsque l’on a une déneigeuse par quartier, ce qui paraît logique. La déneigeuse seule met beaucoup trop de temps et le Soleil irait bien plus vite pour faire disparaître toute cette neige. Enfin la troisième solution, une déneigeuse pour deux quartiers, est efficace mais un peu lente et certains quartiers pourraient se retrouver défavoriser par rapport à d’autres. Il faudrait alors déneiger de nuit, et nous ne sommes pas sûrs que les déneigeurs québécois acceptent ces dures conditions de travail.
